In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from elasticsearch import Elasticsearch, helpers
import configparser
import pandas as pd

config = configparser.ConfigParser()
config.read('example.ini')

es = Elasticsearch(
    cloud_id=config['ELASTIC']['cloud_id'],
    http_auth=(config['ELASTIC']['user'], config['ELASTIC']['password'])
)

# 상명대학교 전자도서관 베스트 셀러 URL
URL="https://libebook.smu.ac.kr/FxLibrary/product/list/?category_type=book&cateopt=best" 

# 총 페이지 수 추출 함수
def get_page_count():
    result = requests.get(URL)

    soup = BeautifulSoup(result.text, "html.parser")

    count = len(soup.select("div.paging span a"))+1
    
    return count

# 각 페이지마다 책 정보 추출 함수
def extract_books(page_count):
    book_list = []
    
    for page in range(page_count):
    
        print(f"Scrapping page {page+1}")
        
        rs = requests.get(f"https://libebook.smu.ac.kr/FxLibrary/product/list/?itemdv=1&sort=3&page={page+1}&itemCount=20&pageCount=10&category=&middlecategory=&cateopt=best&group_num=recommand&catenavi=main&category_type=book&searchoption=&keyoption=&keyoption2=&keyword=&listfilter=all_list&selectview=list_on&searchType=&name=&publisher=&author=&terminal=")
        
        soup = BeautifulSoup(rs.text, "html.parser")
        
        results=soup.select("div.book_list ul.list.typelist li.item")
        
        for rs in results:
            
            try:
                subject=rs.select_one("div.subject a").text
                
                category=rs.select_one("div.category span").text
    
                writer=rs.select("div.info ul.i1 a")[0].text

                publisher=rs.select("div.info ul.i1 a")[1].text

                date=rs.select("div.info ul.i1 li")[2].text
                
                if date=='0000-00-00' or date=="NaN":
                    
                    date='1970-01-01'

                checkout=rs.select("div.info ul.i2 li")[2].text

                like=rs.select("div.info ul.i2 li")[3].text
            
                book_list.append([subject, category, writer, publisher, date, checkout, like])
            
                es.index(
                    index="sm_best_100",
                    document={
                        '제목':subject,
                        'genre':category,
                        'writer':writer,
                        'publisher':publisher,
                        'date':date,
                        'chechout':checkout,
                        'like':like
                    })
                
            except Exception as e:    # 모든 예외의 에러 메시지를 출력할 때는 Exception을 사용
                print('예외가 발생했습니다.', e)
                
            
    return book_list

# CSV 저장
page_count=get_page_count()

books=extract_books(page_count)

df = pd.DataFrame(books, columns = ["제목", "genre", "writer", "pulisher", "date", "checkout","like"])

df.to_csv("sm_best_100", index = False, encoding = 'utf-8-sig')

In [39]:
df

,제목,genre,writer,pulisher,date,checkout,like
0,트렌드 코리아 2023,[경제/비즈니스],"김난도, 전미영, 최지혜, 이수진, 권정윤, 이준영, 이향은, 한다혜, 이혜원, 추예린",미래의창,2022-10-06,누적대출 161,추천 1
1,물고기는 존재하지 않는다,[자연/과학],룰루 밀러,곰출판,2022-01-19,누적대출 191,추천 2
2,달러구트 꿈 백화점,[문학],이미예,북닻,2020-07-28,누적대출 202,추천 4
3,지구 끝의 온실,[문학],김초엽,자이언트북스,2022-04-21,누적대출 135,추천 0
4,불편한 편의점,[문학],김호연,나무옆의자,2022-02-22,누적대출 151,추천 0
...,...,...,...,...,...,...,...
95,커리어 스킬,[컴퓨터/인터넷],존손메즈,길벗,2019-05-10,누적대출 5,추천 0
96,타이탄의 도구들,[경제/비즈니스],팀 페리스,토네이도,2017-04-10,누적대출 161,추천 3
97,픽사 스토리텔링,[경제/비즈니스],매튜 룬(Matthew Luhn),현대지성,2022-02-11,누적대출 14,추천 0
98,핸즈온 머신러닝 (제2판),[강의지원도서],오렐리앙 제롱,한빛미디어,2020-06-03,누적대출 3,추천 0
